# IMDb Scraper

Workflow: 5 

Goal: Start with TMDB ids and get all data from IMDB.

Result: The file ```imdb_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../models')
import imdb_scraper as MB 

## Scrape One Movie

In [4]:
scraper = MB.ImdbScraper()
scraper.open_imdb()

<selenium.webdriver.firefox.webdriver.WebDriver (session="638dbf1f-c617-634d-bccf-78da2e24256d")>


In [5]:
title = 'Kaaliyan'
year = '2017'
# scraper.search_any(title)

In [6]:
# scraper.goto_movie_result(title, year)

In [7]:
# movie_dx = scraper.get_movie_data()
# movie_dx

In [8]:
movie_dx = scraper.get_movie_by_id('tt3170504')
movie_dx 

{'imdb_id': 'tt3170504',
 'title': 'Last Love Lost',
 'original_title': None,
 'year': '2015',
 'rating': 'Not Rated',
 'companies': None,
 'country': None,
 'language': None,
 'duration': 83,
 'directors': 'Jahmar Hill',
 'writers': 'Jahmar Hill',
 'actors': 'Jahmar Hill, Toni Belafonte, Talli Clemons',
 'genres': 'Drama',
 'synopsis': 'Antonio (Jahmar Hill) is awakened from a coma he was in for a year and a half after being shot by his father while trying to break up a domestic dispute between him and his mother. When ',
 'budget': None,
 'gross_us': None,
 'gross_worldwide': None,
 'score': '7.5',
 'votes': '15'}

In [9]:
scraper.close()

## Scrape Based on TMDB

In [14]:
INDEX_FILE = '../data/moviedb_movie.csv'
index_df = PD.read_csv(INDEX_FILE, dtype={'title': str, 'year': str})
index_df.head(3)
index_df.info()

,title,original_title,year,companies,country,language,run_time,crew,cast,poster,genres,collection,synopsis,budget,gross,score,votes,tmdb_id,imdb_id
0,1,1,2013,"Exclusive Media, Flat-Out Films",United States of America,English,112,"Paul Crowder, Mark Monroe, Michael Shevloff","Niki Lauda, Michael Schumacher, Lewis Hamilton",/4uIPXX8DjTsCzUAdtMKHTpojYLq.jpg,Documentary,NaN,Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,7.4,59.0,217316.0,tt2518788
1,"10,000 BC","10,000 BC",2008,"Centropolis Entertainment, Legendary Entertain...",United States of America,English,109,"Roland Emmerich, Sarah Bradshaw, Tom Karnowski","Steven Strait, Camilla Belle, Cliff Curtis",/rnGR3EHkL4ryhQd50XBrtRrV8nq.jpg,"Adventure, Action, Drama, Fantasy",NaN,A prehistoric epic that follows a young mammot...,105000000.0,266000000.0,5.3,1766.0,7840.0,tt0443649
2,1000 Rupee Note,Ek Hazarachi Note,2016,NaN,India,Marathi (Marāṭhī),89,"Shrihari Sathe, Shrikant Bojewar",Sandeep Pathak,/pNNxwXAReV4kh7TCZGqBrl9I72v.jpg,Drama,NaN,Poor Parobudhi receives a thousand rupee note ...,NaN,NaN,6.9,7.0,318654.0,tt2937158
3,1000 To 1,1000 To 1,2014,NaN,United States of America,English,99,Michael Levine,"David Henrie, Hannah Marks, Luke Kleintank",/sGKWrPbYykh6H943XQODN5N8wxi.jpg,Drama,NaN,Cory Weissman is a college basketball player w...,NaN,NaN,5.9,19.0,268245.0,tt2391950
4,100% arabica,100% arabica,1997,NaN,NaN,French,NaN,Mahmoud Zemmouri,NaN,/f5oXyQ1EZqb6gGlKZzxDoI161dQ.jpg,Drama,NaN,"The movie takes place in a poverty-stricken, r...",NaN,NaN,6.0,1.0,99800.0,tt0156245


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197766 entries, 0 to 197765
Data columns (total 19 columns):
title             197766 non-null object
original_title    33925 non-null object
year              33925 non-null object
companies         11603 non-null object
country           13587 non-null object
language          33925 non-null object
run_time          32765 non-null object
crew              32977 non-null object
cast              16017 non-null object
poster            33553 non-null object
genres            16124 non-null object
collection        1189 non-null object
synopsis          33794 non-null object
budget            2865 non-null float64
gross             1910 non-null float64
score             17683 non-null float64
votes             17683 non-null float64
tmdb_id           17683 non-null float64
imdb_id           17471 non-null object
dtypes: float64(5), object(14)
memory usage: 28.7+ MB


In [16]:
# not sure why pandas is creating all these duplicates

doop_df = index_df.duplicated()
doop_df.shape

(17675, 19)

In [11]:
# create jobs

index_1_ df = index_df[0:5349]
index_2_df = index_df[5350:9999]
index_3_df = index_df[10000:13999]
index_4_df = index_df[14000:]

index_1_df.shape
index_2_df.shape
index_3_df.shape
index_4_df.shape

SyntaxError: invalid syntax (<ipython-input-11-119553be072d>, line 3)

In [ ]:
# investigate errors

error_titles = ['13 Times Evil', 'There Is a New World Somewhere', 'The Scarlet and the Black', 'The Sleeping Tiger', 
                'The Untold Story', 'Zombiegeddon', 'To Love The Soul Of A Woman', 'Tree Man']
index_df.loc[index_df['title'].isin(error_titles)][['title', 'year', 'imdb_id']]

In [ ]:
#index_df.tail(20)['title']

In [ ]:
scraper = MB.ImdbScraper()
scraper.open_imdb()
movie_ls = []
t0 = time.time()

for idx, row in index_2_df.iterrows():
    title = row['title']
    year = row['year']
    imdb_id = row['imdb_id']
    movie_dx = None

    try:
        movie_dx = scraper.get_movie_by_id(imdb_id)
    except Exception as ex:
        print('')
        print(f'Error 1: {title} {year} {imdb_id}')
        print(ex)

    if not movie_dx:
        try:
            scraper.open_imdb()
            scraper.search_any(title)
            found = scraper.goto_movie_result(title, year)
            if found:
                print(f'found by title: {title} {year} {imdb_id}')
                movie_dx = scraper.get_movie_data()
            else:
                print(f'NOT found by title: {title} {year}')
        except Exception as ex:
            print(f'Error 2: {title} {year} {imdb_id}')
            print(ex)

    if movie_dx:
        movie_ls.append(movie_dx)

scraper.close()
t1 = time.time()
print(f'movies: {len(movie_ls)}')
print(f'time: {(t1-t0)/60:.2f} mins')

In [ ]:
# projected time

print(f'projected time: {(t1-t0)/60/60*18450/index_df.shape[0]:.2f} hrs')

## Output To CSV

In [ ]:
movie_df = PD.DataFrame(movie_ls)
movie_df[10:15]
movie_df.info()

In [ ]:
save_path = r'../data/imdb_mov_2.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

In [20]:
# duplicates in output file ?

test_1_df = PD.read_csv('../data/imdb_mov_1.csv')
test_1_df.head(3)
test_1_df.shape

,imdb_id,title,original_title,year,rating,companies,country,language,duration,directors,writers,actors,genres,synopsis,budget,gross_us,gross_worldwide,score,votes
0,tt2518788,1,NaN,2013,Not Rated,"Diamond Docs, Exclusive Media Group",USA,English,112.0,Paul Crowder,Mark Monroe,"Michael Fassbender, Niki Lauda, Lewis Hamilton","Documentary, History, Sport",Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,NaN,8.0,3535.0
1,tt0443649,"10,000 BC",NaN,2008,PG-13,"Warner Bros., Legendary Entertainment",USA,English,109.0,Roland Emmerich,"Roland Emmerich, Harald Kloser","Camilla Belle, Steven Strait, Marco Khan","Action, Adventure, Drama","In the prehistoric past, D'Leh is a mammoth hu...",105000000,94784201.0,269784201,5.1,121431.0
2,tt2937158,Ek Hazarachi Note,NaN,2014,Not Rated,Infinitum Productions,India,Marathi,89.0,Shrihari Sathe,Shrikant Bojewar,"Devendra Gaikwad, Usha Naik, Pooja Nayak",Drama,"In this exploration of money and conscience, a...",NaN,2404.0,2404,7.3,309.0


(5364, 19)

In [21]:
test_1_df = test_1_df.drop_duplicates()
test_1_df.shape

(5358, 19)